In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import subprocess

# Utils

In [ ]:
# lambdas
get_range = lambda delta: (np.arange(20) + 1) * delta
file_name = lambda algorithm: f'results/{algorithm}.csv'
get_result = lambda algorithm: pd.read_csv(file_name(algorithm))

In [ ]:
# constant variables
K = 50

low = get_range(10)
high = np.concatenate([get_range(10), get_range(1000)])
ALGORITHMS_RANGES = {
    'insert': low,
    'merge': high,
    'quick': high,
    'dp_quick': high,
    'tim': high
}

In [ ]:
def execute(algorithm, n):
    generate = subprocess.Popen(f'./generate.exe {n} random'.split(), stdout=subprocess.PIPE)
    subprocess.Popen(f'./sort.exe {algorithm}'.split(), stdin=generate.stdout, stdout=subprocess.PIPE)
v_execute = np.vectorize(execute)

def run(algorithm, k):
    for n in ALGORITHMS_RANGES[algorithm]:
        A = np.full(shape=(k,), fill_value=algorithm)
        N = np.full(shape=(k,), fill_value=n)
        v_execute(A, N)


def process_result(df):
    df_cmp = pd.DataFrame(df.groupby('array_size', as_index=False)['comparisons'].mean())
    df_swp = pd.DataFrame(df.groupby('array_size', as_index=False)['swaps'].mean())
    df = pd.merge(df_cmp, df_swp, on='array_size')
    df['total'] = df['comparisons'] + df['swaps']
    return df



def plot_results(results: dict):
    fig = plt.figure(figsize=(16, 10))
    na = len(results) # number of sorting algorithms
    axs = [plt.subplot2grid((2,3), (0,0), colspan=1, rowspan=1), 
           plt.subplot2grid((2,3), (0,1), colspan=1, rowspan=1),
           plt.subplot2grid((2,3), (0,2), colspan=1, rowspan=1),
           plt.subplot2grid((2,3), (1,0), colspan=1, rowspan=1), 
           plt.subplot2grid((2,3), (1,1), colspan=1, rowspan=1),
           plt.subplot2grid((2,3), (1,2), colspan=1, rowspan=1)]
    plt.subplots_adjust(hspace=0.2, wspace=0.25)

    axs[0].set_title(f"Comparison count")
    axs[1].set_title(f"Swaps count")
    axs[2].set_title(f"Total complexity")

    colors = ['blue', 'red', 'green', 'orange', 'purple']
    for (i, (algorithm, df)) in enumerate(results.items()):
        axs[0].plot(df['array_size'], df['comparisons'], color=colors[i], label=algorithm)
        axs[1].plot(df['array_size'], df['swaps'], color=colors[i], label=algorithm)
        axs[2].plot(df['array_size'], df['total'], color=colors[i], label=algorithm)
        axs[3].plot(df.loc[:19, 'array_size'], df.loc[:19, 'comparisons'], color=colors[i], label=algorithm)
        axs[4].plot(df.loc[:19, 'array_size'], df.loc[:19, 'swaps'], color=colors[i], label=algorithm)
        axs[5].plot(df.loc[:19, 'array_size'], df.loc[:19, 'total'], color=colors[i], label=algorithm)


    axs[0].set_ylabel("$cmp(n)$")
    axs[1].set_ylabel("$swp(n)$")
    axs[2].set_ylabel("$cmp(n) + swp(n)$")
    axs[3].set_ylabel("$cmp(n)$")
    axs[4].set_ylabel("$swp(n)$")
    axs[5].set_ylabel("$cmp(n) + swp(n)$")

    for i in range(len(axs)):
        axs[i].set_xlabel("$n$ - array size")
        axs[i].legend()

    plt.show();

# Init necessary files

In [ ]:
# Init result .csv files
for algorithm in ALGORITHMS_RANGES.keys():
    df = pd.DataFrame(columns=['array_size', 'comparisons', 'swaps'])
    df.to_csv(file_name(algorithm), index=False)

In [ ]:
# Init .exe script files
subprocess.run('g++ -o generate generate.cpp -std=c++2a'.split())
subprocess.run('g++ -o sort main.cpp -std=c++2a'.split())

# Run experiments

In [ ]:
print('executing:')
for algorithm in ALGORITHMS_RANGES.keys():
    print(f' -> {algorithm}')
    run(algorithm, K)

In [ ]:
results = {a: get_result(a) for a in ALGORITHMS_RANGES.keys()}
results = {a : process_result(results[a]) for a in results.keys()}

In [ ]:
plot_results(results)